In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import time
import os
import copy
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid

% matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
path=os.getcwd()

In [3]:
df=pd.read_csv(path+'/twins_aligned_cropped_resized/df/pairs_test.csv')

In [4]:
df1=pd.read_csv(path+'/twins_aligned_cropped_resized/df/pairs_train.csv')

In [270]:
def check_overlap(df1,df):
    t=0
    print(df1.shape[0])
    print(df.shape[0])
    for i in range(df1.shape[0]):
        for j in range(df.shape[0]):
            if (df1['id_1'].iloc[i]==df['id_1'].iloc[j] and df1['id_2'].iloc[i]==df['id_2'].iloc[j]) or (df1['id_2'].iloc[i]==df['id_1'].iloc[j] and df1['id_2'].iloc[i]==df['id_1'].iloc[j]): t+=1
    print(t)
    return t

In [271]:
check_overlap(df1,df)

646
208
58


58

## Splitting

In [5]:
df=pd.read_csv(path+'/twins_aligned_cropped_resized/df/pairs.csv')

In [6]:
df['label'].unique()

array(['Same', 'Fraternal', 'Identical', 'UnknownTwinType',
       'IdenticalMirror', 'Sibling', 'IdenticalTriplet'], dtype=object)

In [7]:
df.groupby(['label']).count()

,id_1,id_2
label,,
Fraternal,32,32
Identical,348,348
IdenticalMirror,18,18
IdenticalTriplet,6,6
Same,432,432
Sibling,8,8
UnknownTwinType,10,10


In [8]:
df[(df['label']=='Sibling') ]

,id_1,id_2,label
566,90142,90144,Sibling
568,90143,90144,Sibling
570,90144,90142,Sibling
571,90144,90143,Sibling
604,90201,90202,Sibling
605,90201,90203,Sibling
606,90202,90201,Sibling
608,90203,90201,Sibling


In [9]:
#{'id_1':[0],'id_2':[0],'label':[0]}
train=pd.DataFrame({})
test=pd.DataFrame({})

same=df[df['label']=='Same']
ident=df[(df['label']=='Identical')| (df['label']=='IdenticalMirror')
         | (df['label']=='UnknownTwinType') |(df['label']=='Fraternal')|(df['label']=='Sibling')]

other=df[(df['label']=='IdenticalTriplet') ]#| (df['label']=='Sibling')

print(same.shape)
print(ident.shape)
print(other.shape)

(432, 3)
(416, 3)
(6, 3)


In [10]:
train=train.append(other)

In [13]:
def inverse(a):
    copy=a.copy()
    #print(a)
    a['id_1']=str(copy['id_2'])
    a['id_2']=str(copy['id_1'])
    #print(a)
    return a

while ident.shape[0]>0:
    prob=np.random.randint(0,11)
    t=ident.iloc[np.random.randint(ident.shape[0])]
    if prob>=8:
        test=test.append(t)
        test=test.append(inverse(t))
    else:
        train=train.append(t)
        train=train.append(inverse(t))
    ident=ident[(ident['id_1']!=t['id_1']) & (str(ident['id_2'])!=str(t['id_2'])) ]
    ident=ident[(ident['id_1']!=inverse(t)['id_1']) &(ident['id_2']!=inverse(t)['id_2']) ]
print(test.shape)
print(train.shape)
check_overlap(test,train)

KeyError: True

In [307]:
while same.shape[0]>0:
    t=same.iloc[np.random.randint(same.shape[0])] 
    if t['id_1'] in train['id_1'].tolist():
        train=train.append(t)
    else:
        test=test.append(t)
    same=same[(same['id_1']!=t['id_1']) &(same['id_2']!=t['id_2']) ]
print(test.shape)
print(train.shape)
check_overlap(test,train)

(431, 3)
(9, 3)
431
9
0


0

In [308]:
print(test.shape)
print(train.shape)
check_overlap(test,train)

(431, 3)
(9, 3)
431
9
0


0

In [31]:
train.to_csv(path+'/twins_aligned_cropped_resized/df/pairs_train_fedor.csv',index =False)
test.to_csv(path+'/twins_aligned_cropped_resized/df/pairs_test_fedor.csv',index =False)

## Read

In [32]:
train=pd.read_csv(path+'/twins_aligned_cropped_resized/df/pairs_train_fedor.csv')
test=pd.read_csv(path+'/twins_aligned_cropped_resized/df/pairs_test_fedor.csv')

In [33]:
test.head()

,id_1,id_2,label
0,90018,90019,Identical
1,90019,90018,Identical
2,90152,90151,Identical
3,90151,90152,Identical
4,90248,90249,Identical


In [29]:
test['id_1']=test['id_1'].apply(int)
test['id_2']=test['id_2'].apply(int)
train['id_1']=train['id_1'].apply(int)
train['id_2']=train['id_2'].apply(int)

In [30]:
test.head()

,id_1,id_2,label
0,90018,90019,Identical
1,90019,90018,Identical
2,90152,90151,Identical
3,90151,90152,Identical
4,90248,90249,Identical
